# Attemp number 2
This attemp includes findings from my previous attemp, bringing them together to form a more refined approach at producing a fully cleaned dataset

In [1]:
import pandas as pd
import numpy as np
from pathlib import Path

script_path = Path.cwd()
data_path_cleaned = script_path.parent / "data" / "games_cleaned.csv"

df = pd.read_csv(data_path_cleaned, encoding='utf-8')

# Dtatype conversion
df['Release date'] = pd.to_datetime(df['Release date'], errors='coerce')
print(df.dtypes)

AppID                                int64
Name                                object
Release date                datetime64[ns]
Estimated owners                    object
Required age                         int64
Price                              float64
User score                           int64
Positive                             int64
Negative                             int64
Recommendations                      int64
Average playtime forever             int64
Developers                          object
Publishers                          object
Categories                          object
Genres                              object
Tags                                object
dtype: object


## Estimated average owners

In [2]:
# Split 'Estimated owners' into two columns and calculate the average
owners_clean = df['Estimated owners'].str.replace(' ', '', regex=False)

owners_split = owners_clean.str.split('-', expand=True)

owners_split[0] = pd.to_numeric(owners_split[0], errors='coerce')
owners_split[1] = pd.to_numeric(owners_split[1], errors='coerce')

df['Estimated average owners'] = (owners_split[0] + owners_split[1]) / 2
df['Estimated average owners'] = df['Estimated average owners'].astype('Int64')

print(df[['Estimated owners', 'Estimated average owners']].head(10))

  Estimated owners  Estimated average owners
0        0 - 20000                     10000
1        0 - 20000                     10000
2        0 - 20000                     10000
3        0 - 20000                     10000
4        0 - 20000                     10000
5   50000 - 100000                     75000
6        0 - 20000                     10000
7        0 - 20000                     10000
8        0 - 20000                     10000
9   50000 - 100000                     75000


## Reranging the dataset

In [3]:
# Adding new column for lowercase names
df['Name_lowercase'] = df['Name'].str.lower().str.strip()

# Check for duplicates
duplicates = df[df['Name_lowercase'].duplicated(keep=False)]
print(duplicates.shape)

(2655, 18)


### Bayesian average
As we move beyond simple ratios, using only the share of positive reviews can be misleading since it ignores how many reviews support that score. A game with 9 out of 10 positives looks the same as one with 9,000 out of 10,000, even though the latter is far more reliable.

The `Bayesian average` addresses this by blending a game’s observed positive ratio with a global positive ratio. It does this by adding a fixed number of imaginary reviews to the total, which reduces the influence of small sample sizes. As the number of real reviews grows, the Bayesian average moves closer to the game’s actual ratio. This results in a more stable and reliable estimate of quality.
$$
\text{Bayesian average} \;=\; \frac{{p}\,·n + \mu\,·m}{n + m}
$$
**Where:**
- **$p$ = Positive ratio**
- **$n$ = Number of reviews**
- **$\mu$ = Global positive ratio**
- **$m$ = Number of additional imaginary reviews**

In [4]:
# Bayesian average
df["n_reviews"] = df["Positive"] + df["Negative"]
df["positive_ratio"] = df["Positive"] / df["n_reviews"].replace(0, np.nan)

# Summary statistics for the amount of reviews
print("Mean:", df["n_reviews"].mean())
print("Median:", df["n_reviews"].median())
print("Min:", df["n_reviews"].min())
print("Max:", df["n_reviews"].max())
print("Quantiles:", df["n_reviews"].quantile([0.25,0.5,0.75,0.9,0.99]))


Mean: 880.2117054875641
Median: 5.0
Min: 0
Max: 6531097
Quantiles: 0.25        0.0
0.50        5.0
0.75       39.0
0.90      293.0
0.99    11618.0
Name: n_reviews, dtype: float64


In [5]:

# Global postive ratio
mu = df["Positive"].sum() / df["n_reviews"].sum()

# Bayesian score function
def bayesian_score(pos_ratio, n, mu, m):
    return ((pos_ratio * n + mu * m) / (n + m)).fillna(mu)

# Compute for different m values
for m in [25,50,100]:
    df[f"bayes_m{m}"] = bayesian_score(df["positive_ratio"], df["n_reviews"], mu, m)

df_filtered = df[df["n_reviews"] >= 20].copy()

for m in [25,50,100]:
    print(f"\nm={m}")
    print(
        df_filtered.sort_values(by=['n_reviews', f"bayes_m{m}"], ascending=[True,False])
        [["n_reviews","Positive", "Negative", "positive_ratio", f"bayes_m{m}"]]
        .head(5), "\n",
        df_filtered.sort_values(by=['n_reviews', f"bayes_m{m}"], ascending=[True,True])
        [["n_reviews","Positive", "Negative", "positive_ratio", f"bayes_m{m}"]]
        .head(5)
    )


m=25
      n_reviews  Positive  Negative  positive_ratio  bayes_m25
1348         20        20         0             1.0   0.920563
3191         20        20         0             1.0   0.920563
4023         20        20         0             1.0   0.920563
4037         20        20         0             1.0   0.920563
4259         20        20         0             1.0   0.920563 
        n_reviews  Positive  Negative  positive_ratio  bayes_m25
7135          20         2        18            0.10   0.520563
18418         20         2        18            0.10   0.520563
46857         20         3        17            0.15   0.542785
49523         20         3        17            0.15   0.542785
50612         20         3        17            0.15   0.542785

m=50
      n_reviews  Positive  Negative  positive_ratio  bayes_m50
1348         20        20         0             1.0   0.897866
3191         20        20         0             1.0   0.897866
4023         20        20         0

#### m = 50
The analysis reveals a **highly skewed** distribution of review counts, with a **median of only 5 and an upper quartile of 39 (also at 90% = 293)**, when put into comparison with the **mean of 880 when having a 99% of  11618 (max of 6531097)**, it is determined that the m will not be able to be a big number as a histogram will highlight the dominance of a long tail of sparsely reviewed titles.  
  
Under this imbalance, `m = 50` was selected, representing the number of additional imaginary reviews incorporated into the calculation of Bayesian average. This choice mitigates the disproportionate influence of extremely small sample sizes while preserving the integrity of ratings derived from large, well supported review counts, thereby achieving a balance between stability and sensitivity across the dataset.


In [6]:
# Selecting m as 50
m = 50
df["bayes_average"] = bayesian_score(df["positive_ratio"], df["n_reviews"], mu, m)

df_sorted = df.sort_values(
    by=['Estimated average owners', 'bayes_average', 'n_reviews'],
    ascending=[False, False, False]
)

print(df_sorted[['Estimated average owners', 'bayes_average', 'positive_ratio', 'n_reviews']].head(10))

       Estimated average owners  bayes_average  positive_ratio  n_reviews
17585                 150000000       0.830987        0.830986    1777590
96948                  75000000       0.958507        0.958515     691809
30583                  75000000       0.935611        0.935615     880376
46158                  75000000       0.882611        0.882611    6531097
7030                   75000000       0.677070        0.677030     228814
8885                   75000000       0.563080        0.563072    2050633
57459                  35000000       0.980546        0.980557     533257
46416                  35000000       0.978652        0.978658     986027
39530                  35000000       0.974499        0.974508     635661
1289                   35000000       0.965925        0.965931     851330


### Missing values

In [7]:
# Initial exploration of missing values
print(df.isnull().sum())

AppID                           0
Name                            6
Release date                  131
Estimated owners                0
Required age                    0
Price                           0
User score                      0
Positive                        0
Negative                        0
Recommendations                 0
Average playtime forever        0
Developers                   6475
Publishers                   6778
Categories                   7566
Genres                       6440
Tags                        37423
Estimated average owners        0
Name_lowercase                  6
n_reviews                       0
positive_ratio              37588
bayes_m25                       0
bayes_m50                       0
bayes_m100                      0
bayes_average                   0
dtype: int64


In [8]:
# Number of missing values in major columns that can be found in duplicates
columns_to_check = ['Developers', 'Publishers', 'Categories', 'Genres', 'Tags']

for col in columns_to_check:
    missing_rows = df[df[col].isnull()]
    
    can_be_filled = missing_rows['Name_lowercase'].isin(
        df[~df[col].isnull()]['Name_lowercase']
    )
    
    fillable_rows = missing_rows[can_be_filled]
    
    print(f"{col}: {len(fillable_rows)} rows can be filled from duplicates.")


Developers: 16 rows can be filled from duplicates.
Publishers: 26 rows can be filled from duplicates.
Categories: 31 rows can be filled from duplicates.
Genres: 13 rows can be filled from duplicates.
Tags: 595 rows can be filled from duplicates.



To fill the datas across duplicated game entries, I grouped rows by `Name_lowercase` and updated using `.update` with `.ffill` and `.bfill` to propagate only missing values within each group. This way, no matter which duplicate held the information, all rows for the same game now share the filled values


In [9]:
# Filling missing values from the duplicates
cols_to_fill = ['Developers', 'Publishers', 'Categories', 'Genres', 'Tags']

for col in cols_to_fill:
    df.update(df.groupby('Name_lowercase')[col].ffill())
    df.update(df.groupby('Name_lowercase')[col].bfill())


In [10]:
# Double checking the missing values after filling
cols_to_check = ['Developers', 'Publishers', 'Categories', 'Genres', 'Tags']

for col in cols_to_check:
    missing_rows = df[df[col].isnull()]
    
    can_be_filled = missing_rows['Name_lowercase'].isin(
        df[~df[col].isnull()]['Name_lowercase']
    )
    
    fillable_rows = missing_rows[can_be_filled]
    
    print(f"{col}: {len(fillable_rows)} rows can be filled from duplicates.")


Developers: 0 rows can be filled from duplicates.
Publishers: 0 rows can be filled from duplicates.
Categories: 3 rows can be filled from duplicates.
Genres: 2 rows can be filled from duplicates.
Tags: 0 rows can be filled from duplicates.


After filling missing data within duplicate groups, 5 rows are still empty, althought is being detected as the duplicate contains the much needed information, but my .update approach does not seem to work for this specific situations.

This may be because of 1. small differences in how the groups were formed or 2. because the update step did not replace the missing values.  
Since the number is small, the next step is to check these rows directly or combine the duplicates into a single row per game.

In [11]:
print(df.isnull().sum())

AppID                           0
Name                            6
Release date                  131
Estimated owners                0
Required age                    0
Price                           0
User score                      0
Positive                        0
Negative                        0
Recommendations                 0
Average playtime forever        0
Developers                   6459
Publishers                   6752
Categories                   7538
Genres                       6429
Tags                        36828
Estimated average owners        0
Name_lowercase                  6
n_reviews                       0
positive_ratio              37588
bayes_m25                       0
bayes_m50                       0
bayes_m100                      0
bayes_average                   0
dtype: int64


In [12]:
null_rows = df[df['positive_ratio'].isnull()]
print(null_rows[['Positive', 'Negative', 'positive_ratio', 'bayes_average']])

        Positive  Negative  positive_ratio  bayes_average
2              0         0             NaN       0.857013
7              0         0             NaN       0.857013
18             0         0             NaN       0.857013
29             0         0             NaN       0.857013
43             0         0             NaN       0.857013
...          ...       ...             ...            ...
111442         0         0             NaN       0.857013
111444         0         0             NaN       0.857013
111448         0         0             NaN       0.857013
111450         0         0             NaN       0.857013
111451         0         0             NaN       0.857013

[37588 rows x 4 columns]


In [13]:
df = df.drop(columns=['bayes_m25','bayes_m50', 'bayes_m100'])

# Only 6 rows still have missing values, therefore dropping them might be the better option. 
# By dropping these rows, we should also be dropping the rows with missing values in Name_lowercase
df = df.dropna(subset=['Name'])

#filling remaining string columns with 'Unknown'
df['Release date'] = df['Release date'].fillna('Unknown')
df['Developers'] = df['Developers'].fillna('Unknown')
df['Publishers'] = df['Publishers'].fillna('Unknown')
df['Categories'] = df['Categories'].fillna('Unknown')
df['Genres'] = df['Genres'].fillna('Unknown')
df['Tags'] = df['Tags'].fillna('Unknown')

# Filling remaining numeric columns with 0
df['Estimated average owners'] = df['Estimated average owners'].fillna(0)
df['positive_ratio'] = df['positive_ratio'].fillna(0)

print(df.isnull().sum())

AppID                       0
Name                        0
Release date                0
Estimated owners            0
Required age                0
Price                       0
User score                  0
Positive                    0
Negative                    0
Recommendations             0
Average playtime forever    0
Developers                  0
Publishers                  0
Categories                  0
Genres                      0
Tags                        0
Estimated average owners    0
Name_lowercase              0
n_reviews                   0
positive_ratio              0
bayes_average               0
dtype: int64


### Duplicates

In [14]:
# Sorting by the priorities
df_sorted = df.sort_values(
    by=['Estimated average owners', 'bayes_average', 'n_reviews'],
    ascending=[False, False, False]
)

# Keeping the first duplicate as its being sorted already
df_drop = df_sorted.drop_duplicates('Name_lowercase', keep='first')

data_path_final = script_path.parent / "data" / "games_final.csv"
df_drop.to_csv(data_path_final, index=False, encoding="utf-8")

print("cleaned")

cleaned


In [15]:
# Double checking the duplicates in the final cleaned file
df2 = pd.read_csv(data_path_final, encoding='utf-8')

duplicates2 = df2[df2['Name_lowercase'].duplicated(keep=False)]
print(duplicates2.shape)

(0, 21)
